In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,33282
2,Huelva,Confirmados PDIA 14 días,985
3,Huelva,Tasa PDIA 14 días,"191,9441900344915"
4,Huelva,Confirmados PDIA 7 días,419
5,Huelva,Total Confirmados,33466
6,Huelva,Curados,31530
7,Huelva,Fallecidos,389


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  33282.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  9102.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1115 personas en los últimos 7 días 

Un positivo PCR cada 413 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,33282.0,419.0,985.0,513170.0,81.649356,191.944190,76.0
Huelva-Costa,19162.0,228.0,542.0,289548.0,78.743421,187.188307,49.0
Condado-Campiña,10833.0,163.0,357.0,156231.0,104.332687,228.507787,23.0
Huelva (capital),9102.0,129.0,348.0,143837.0,89.684852,241.940530,11.0
Moguer,1406.0,39.0,68.0,21867.0,178.350940,310.970869,10.0
Cartaya,1404.0,29.0,46.0,20083.0,144.400737,229.049445,10.0
Aljaraque,1171.0,12.0,24.0,21474.0,55.881531,111.763062,8.0
Lepe,2175.0,14.0,29.0,27880.0,50.215208,104.017217,5.0
Isla Cristina,2187.0,13.0,28.0,21393.0,60.767541,130.883934,5.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
San Juan del Puerto,586.0,24.0,48.0,9411.0,255.020720,510.041441,2.0
Beas,232.0,6.0,19.0,4341.0,138.217001,437.687169,0.0
Puebla de Guzmán,141.0,2.0,12.0,3092.0,64.683053,388.098318,0.0
Cerro de Andévalo (El),154.0,6.0,9.0,2327.0,257.842716,386.764074,0.0
Almendro (El),39.0,1.0,3.0,840.0,119.047619,357.142857,0.0
Valverde del Camino,664.0,11.0,43.0,12750.0,86.274510,337.254902,1.0
Palos de la Frontera,729.0,22.0,39.0,11742.0,187.361608,332.141032,4.0
Calañas,201.0,1.0,9.0,2768.0,36.127168,325.144509,0.0
Moguer,1406.0,39.0,68.0,21867.0,178.350940,310.970869,10.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Calañas,201.0,1.0,9.0,2768.0,36.127168,325.144509,0.0,0.111111
Puebla de Guzmán,141.0,2.0,12.0,3092.0,64.683053,388.098318,0.0,0.166667
Villablanca,192.0,1.0,5.0,2885.0,34.662045,173.310225,1.0,0.200000
San Bartolomé de la Torre,307.0,1.0,5.0,3761.0,26.588673,132.943366,0.0,0.200000
Lucena del Puerto,267.0,2.0,8.0,3261.0,61.330880,245.323520,0.0,0.250000
Valverde del Camino,664.0,11.0,43.0,12750.0,86.274510,337.254902,1.0,0.255814
Almonte,1916.0,15.0,55.0,24507.0,61.207002,224.425674,1.0,0.272727
Aracena,541.0,2.0,7.0,8255.0,24.227741,84.797093,0.0,0.285714
Niebla,249.0,2.0,7.0,4116.0,48.590865,170.068027,0.0,0.285714
